In [19]:

def process(cap):
    
    global net, classes, layer_names, output_layers, saveVid
    saveVidFlag = saveVid.get() 
    f_x,f_y = 0.8,0.8
    
    if saveVidFlag:
        ret,frame = cap.read()
        frame = cv2.resize(frame, (0, 0), fx=f_x, fy=f_y)

        size = (frame.shape[1],frame.shape[0])

        result = cv2.VideoWriter('processed.avi', cv2.VideoWriter_fourcc(*'MJPG'), 15, size)

    flag = True
    while cap.isOpened():
       
        if flag:
            ret, frame = cap.read()
            if ret==False:
                break
            frame=cv2.resize(frame,(0,0),fx=f_x,fy=f_y)
            
            height, width, channels = frame.shape

            # Detecting objects
            blob = cv2.dnn.blobFromImage(frame,scalefactor= 0.00392, size=(320,320), mean=(0, 0, 0), swapRB=True, crop=False)#0.00392

            net.setInput(blob)

            outs = net.forward(output_layers)
            
            # Showing informations on the screen
            class_ids = []
            confidences = []
            boxes = []

            for out in outs:
                for detection in out:
                    
                    scores = detection[5:]
                    class_id = np.argmax(scores)
                    confidence = detection[4]

                    if confidence > 0.5:
                        
                        # Object detected
                        center_x = int(detection[0] * width)
                        center_y = int(detection[1] * height)
                        w = int(detection[2] * width)
                        h = int(detection[3] * height)

                        # Rectangle coordinates
                        x = int(center_x - w / 2)
                        y = int(center_y - h / 2)

                        boxes.append([x, y, w, h])
                        confidences.append(float(confidence))
                        class_ids.append(class_id)

            indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
            
            if len(indexes)==1:
                indexes = indexes[0]
            else:
                indexes = np.squeeze(indexes)


            
            font = cv2.FONT_HERSHEY_PLAIN
            
            
            for i in indexes:
                x, y, w, h = boxes[i]
                label = str(classes[class_ids[i]])        
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0,255,0), 2)
                cv2.putText(frame, label+"  "+str(round(confidences[i],2)*100)+"%", (x, y + 10), font, 1.2, (0,0,255), 2)
                
            if saveVidFlag:
                result.write(frame)
            

            cv2.imshow("Yolov3 Object Detection", frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        flag = not flag
    if saveVidFlag:
        result.release()

    cap.release()
    cv2.destroyAllWindows()

In [11]:
def video(pth):    
    
    cap = cv2.VideoCapture(pth)
    
    if cap.isOpened() == False:
        print("Video Error")
    else:
        process(cap)

In [12]:
def camera():
    cap = cv2.VideoCapture(0)
    if cap.isOpened() == False:
        print("Camera Error")
    else:
        process(cap)
    
    


In [22]:
import numpy as np
import cv2

import tkinter
from tkinter import filedialog

base = tkinter.Tk()
base.geometry('500x400')

    
##########################################
#please specify the names of your files (copy of yolov3.weights or copy of yolov3.cfg)
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")

net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)
classes = []

with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

layer_names = net.getLayerNames()


output_layers = [layer_names[i - 1] for i in np.squeeze(net.getUnconnectedOutLayers())]


t = Label(base, text = "Object Recognition Using openCV") 
t.config(font =("Courier", 18))
t.pack()

x = Button(base, text ='Camera', height = 3, bg="#61ffc8", width = 20,command = lambda:camera())
x.pack()
y = Button(base, text ='Select a file', height = 3, bg="#61ffc8",width = 20,command = lambda:video(filedialog.askopenfile(initialdir="/").name))
y.pack()
saveVid = IntVar()
C1 = Checkbutton(base, text="Save?", variable=saveVid, onvalue = 1,offvalue = 0,height = 2, width = 30)
C1.pack()


base.mainloop()


